# ROME PROJECT
###### Daniel Elechiguerra Batlle

I have coded the general parameters so that only the cells that have assignments need to be changed.

As for the algorithms, changing the parameters that appear at the beginning of the cell should be enough and it should run automatically without the need to change anything else. You can also optimize the hyperparameters by: **Grid Search, Random Search, Bayesian Optimization y Genetic Algorithms**.

Parameters and results for **MLP, GRU, RNN y LSTM (y BI-LSTM)** are automatically logged in **"Results1.xlsx"** in order to being able of comparing results. **CNN** usses different hyperparameters and should be saved independently. 
Currently, I am saving (but this information could be expanded if necessary):


> *Algorithm,	Layers,	Neurons per layer,	Activation,	Optimizer,	Learning rate, Horizon,	Back steps,	Batch size,	Epochs,	Dataset,	Learning MSE,	Validation MSE,	Traning time*


##### Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import sklearn
import time
import re
import random as rn
import json
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, GRU, SimpleRNN, Conv1D, Flatten, Dropout, MaxPooling1D, Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


##### Select working dataset

In [ ]:
dataset = '1h' # Choose 15min (more samples) or 1h (less samples)

##### Aux Functions (No debería hacer falta modificar nada aquí)

In [ ]:
# This function transforms the input set into 2D/3D inputs
# 3D: (samples, back_steps, features)
# 2D: (samples, back_steps*features)

def create_dataset(old_x, old_y, back_steps = 1, horizon = 1, task = '3d'):
    
    new_x, new_y = pd.DataFrame(), pd.DataFrame()
    for i in range(1, back_steps + 1):
        columns = []
        for column in list(old_x.columns):
            columns.append(column + ' - ' + str(i))
        new_x[columns] = old_x.shift(i)
        new_x['y - ' + str(i)] = old_y.shift(i)
    new_x = new_x[back_steps + horizon - 1:].reset_index(drop = True)
    new_y = old_y[back_steps + horizon - 1:].reset_index(drop = True)
    if task == '3d':
        return np.array(new_x).reshape((np.array(new_x).shape[0], back_steps, int(np.array(new_x).shape[1]/back_steps))), np.array(new_y)
    elif task == '2d':
        return new_x, new_y
    
    return

In [ ]:
# Load data (careful with the loading if next function is moved to a different script)

data = pd.read_csv('TTOTdataEEM17v3.csv', index_col = 0).reset_index(drop = True)
data['Date'] = pd.to_datetime(data['Date'])

if dataset == '15min': 
    data.drop(['Date', 'Hour', 'Min'], axis = 1, inplace = True)
    x = data.iloc[:,:-1]
    x = x[x.columns[1::3]]
    y = pd.DataFrame(data.iloc[:,-1])
else: 
    data_hour = data.groupby(by = ['Date', 'Hour']).mean()
    data_hour = data_hour.reset_index(drop = True)
    x = data_hour.iloc[:,:-1]
    x = x[x.columns[1::3]]
    y = pd.DataFrame(data_hour.iloc[:,-1])
    
def prepare_data(task = '3d', back_steps = 1, horizon = 1, valid_size = 0.2, shuffle = False):

    # Train / Valid
    if shuffle:
        x_train, x_valid, y_train, y_valid = train_test_split(x, y, valid_size = valid_size, random_state = 42)
    else:
        x_train = x.iloc[:int((1 - valid_size)*len(x)),:]
        x_valid = x.iloc[int((1 - valid_size)*len(x)):,:]
        y_train = y.iloc[:int((1 - valid_size)*len(y)),:]
        y_valid = y.iloc[int((1 - valid_size)*len(y)):,:].reset_index(drop = True)

    # Scale input
    scaler = StandardScaler()
    x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
    x_valid = pd.DataFrame(scaler.transform(x_valid), columns = x_valid.columns)
    
    # Scale output
    
    # Create dataset
    if task == '3d':
        x_train, y_train = create_dataset(x_train, y_train, back_steps, horizon, '3d')
        x_valid, y_valid = create_dataset(x_valid, y_valid, back_steps, horizon, '3d')
    elif task == '2d':
        x_train, y_train = create_dataset(x_train, y_train, back_steps, horizon, '2d')
        x_valid, y_valid = create_dataset(x_valid, y_valid, back_steps, horizon, '2d')
    
    return x_train, y_train, x_valid, y_valid

In [ ]:
seed = 1234

def set_seed(seed = seed):
    
    import os
    import tensorflow as tf

    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(seed)
    rn.seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    tf.random.set_seed(1234)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

###### Select General Parameters 

In [ ]:
# General Parameters
back_steps = 10
horizon = 1

##### Benchmark MSE

In [ ]:
# Error of predicting the same output as the last interval value can be used as benchmark

x_train, y_train, x_valid, y_valid = prepare_data(task = '2d', back_steps = back_steps, horizon = horizon)
mse_benchmark = sklearn.metrics.mean_squared_error(y_valid[horizon:], y_valid[:y_valid.shape[0] - horizon])
print('Benchmark MSE:', mse_benchmark)

Benchmark MSE: 272.83736866408935


### Hyperparameter Optimization 
###### (todas las celdas anteriores son necesarias)

#### Define Models 
###### (esta celda hay que correrla para todos los tipos de optimización)

##### MLP, GRU, RNN, LSTM, BI LSTM

In [ ]:
def create_model(algorithm, activation, optimizer, learning_rate, layers, neurons_per_layer, shape):

    # Layer hyperparameters and number of layers  
    neurons_per_layer = [int(neurons_per_layer)]*int(layers)  

    # Optimizer
    if optimizer == 'adam':
      opt = keras.optimizers.Adam(lr = learning_rate)
    elif optimizer == 'nadam':
      opt = keras.optimizers.Nadam(lr = learning_rate)
    elif optimizer == 'adadelta':
      opt = keras.optimizers.Adadelta(lr = learning_rate)
    elif optimizer == 'adagrad':
      opt = keras.optimizers.Adagrad(lr = learning_rate)
    elif optimizer == 'rmsprop':
      opt = keras.optimizers.RMSprop(lr = learning_rate)

    ## MODEL
    model = Sequential()

    if algorithm == 'mlp':
      # Add layers
      model.add(Dense(neurons_per_layer[0], input_dim = shape[1], activation = activation)) 
      for neurons in neurons_per_layer[1:]:
          model.add(Dense(neurons, activation = activation))
      model.add(Dense(1))

    elif algorithm == 'rnn':
      # Add layers
      if len(neurons_per_layer) == 1:
          model.add(SimpleRNN(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = False, activation = activation))
      else:
          model.add(SimpleRNN(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = True, activation = activation))
          for neurons in neurons_per_layer[1:-1]:
              model.add(SimpleRNN(neurons, return_sequences = True, activation = activation))
          model.add(SimpleRNN(neurons_per_layer[-1], return_sequences = False, activation = activation))
      model.add(Dense(1))

    elif algorithm == 'gru':
      # Add layers
      if len(neurons_per_layer) == 1:
          model.add(GRU(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = False, activation = activation))
      else:
          model.add(GRU(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = True, activation = activation))
          for neurons in neurons_per_layer[1:-1]:
              model.add(GRU(neurons, return_sequences = True, activation = activation))
          model.add(GRU(neurons_per_layer[-1], return_sequences = False, activation = activation))
      model.add(Dense(1))

    elif algorithm == 'lstm':
      # Add layers
      if len(neurons_per_layer) == 1:
          model.add(LSTM(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = False, activation = activation))
      else:
          model.add(LSTM(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = True, activation = activation))
          for neurons in neurons_per_layer[1:-1]:
              model.add(LSTM(neurons, return_sequences = True, activation = activation))
          model.add(LSTM(neurons_per_layer[-1], return_sequences = False, activation = activation))
      model.add(Dense(1))

    elif algorithm == 'bilstm':
      # Add layers
      if len(neurons_per_layer) == 1:
          model.add(Bidirectional(LSTM(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = False, activation = activation)))
      else:
          model.add(Bidirectional(LSTM(neurons_per_layer[0], input_shape = (shape[1], shape[2]), return_sequences = True, activation = activation)))
          for neurons in neurons_per_layer[1:-1]:
              model.add(Bidirectional(LSTM(neurons, return_sequences = True, activation = activation)))
          model.add(Bidirectional(LSTM(neurons_per_layer[-1], return_sequences = False, activation = activation)))
      model.add(Dense(1))

    # Compile model
    model.compile(loss = 'mean_squared_error', optimizer = opt)

    return model

#### Grid Search

##### GRU, RNN, LSTM, BI LSTM

In [ ]:
# Hiperparámetros a optimizar. Se pueden seleccionar uno o varios algoritmos!
grid = {'algorithm' : ['gru'], #['gru', 'rnn', 'lstm', 'bilstm'],
            'activation' : ['elu'], #['elu', 'relu', 'selu']
            'layers' : list(np.arange(1,5,1)), #min_layers, max_layers (no incluido), step_size
            'neurons_per_layer' : list(np.arange(16, 28, 4)), #min_neurons, max_neurons (no incluido), step_size
            'learning_rate' : list(np.exp(np.linspace(np.log(0.0005),np.log(0.005), 5))), #min_lr, max_lr (incluido), number_of_elements
            'optimizer' : ['adam'], #['adam', 'nadam', 'adadelta', 'adagrad', 'rmsprop']
            'batch_size' : [20],
            'epochs' : [20]}






x_train, y_train, x_valid, y_valid = prepare_data(task = '3d', back_steps = back_steps, horizon = horizon)

# create model
model = KerasRegressor(build_fn = create_model, shape = x_train.shape, verbose = 0)

grid_model = GridSearchCV(estimator = model, param_grid = grid, verbose = 1, refit = False, n_jobs = -1, pre_dispatch = '2*n_jobs', scoring = 'neg_mean_squared_error',  return_train_score = True, 
                          cv = [(list(np.arange(0,x_train.shape[0],1)),list(np.arange(x_train.shape[0], x_train.shape[0]+x_valid.shape[0],1)))])
grid_result = grid_model.fit(np.concatenate((x_train, x_valid)),np.concatenate((y_train, y_valid)))

# save results
results = pd.read_excel('Results1.xlsx')
for i in range(0,len(grid_model.cv_results_['mean_test_score'])):
  results.loc[len(results)] = [grid_model.cv_results_['params'][i]['algorithm'], int(grid_model.cv_results_['params'][i]['layers']), int(grid_model.cv_results_['params'][i]['neurons_per_layer']), 
                              grid_model.cv_results_['params'][i]['activation'], grid_model.cv_results_['params'][i]['optimizer'], grid_model.cv_results_['params'][i]['learning_rate'], 
                              horizon, back_steps, grid_model.cv_results_['params'][i]['batch_size'],grid_model.cv_results_['params'][i]['epochs'], dataset, -grid_model.cv_results_['mean_train_score'][i], -grid_model.cv_results_['mean_test_score'][i], grid_model.cv_results_['mean_fit_time'][i]]
results.to_excel('Results1.xlsx', index = False)

print('Best MSE: ', -grid_model.best_score_)
print('Params: ', grid_model.best_params_)

Fitting 1 folds for each of 8 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  5.9min finished


Best MSE:  172.14781082137594
Params:  {'activation': 'elu', 'algorithm': 'gru', 'layers': 2, 'learning_rate': 0.0005000000000000001, 'neurons_per_layer': 24, 'optimizer': 'adam'}


##### MLP

In [ ]:
# Hiperparámetros a optimizar. Se pueden seleccionar uno o varios algoritmos!
grid = {'algorithm' : ['mlp'],
            'activation' : ['elu'], #['elu', 'relu', 'selu', ]
            'layers' : list(np.arange(1,5,1)), #min_layers, max_layers (no incluido), step_size
            'neurons_per_layer' : list(np.arange(16, 28, 4)), #min_neurons, max_neurons (no incluido), step_size
            'learning_rate' : list(np.exp(np.linspace(np.log(0.0005),np.log(0.005), 5))), #min_lr, max_lr (incluido), number_of_elements
            'optimizer' : ['adam'], #['adam', 'nadam', 'adadelta', 'adagrad', 'rmsprop']
            'batch_size' : [20],
            'epochs' : [20]}





x_train, y_train, x_valid, y_valid = prepare_data(task = '2d', back_steps = back_steps, horizon = horizon)

# create model
model = KerasRegressor(build_fn = create_model, shape = x_train.shape, verbose = 0)

grid_model = GridSearchCV(estimator = model, param_grid = grid, verbose = 1, refit = False, n_jobs = -1, pre_dispatch = '2*n_jobs', scoring = 'neg_mean_squared_error', return_train_score = True,
                          cv = [(list(np.arange(0,x_train.shape[0],1)),list(np.arange(x_train.shape[0], x_train.shape[0]+x_valid.shape[0],1)))])
grid_result = grid_model.fit(np.concatenate((x_train, x_valid)),np.concatenate((y_train, y_valid)))

# save results
results = pd.read_excel('Results1.xlsx')
for i in range(0,len(grid_model.cv_results_['mean_test_score'])):
  results.loc[len(results)] = [grid_model.cv_results_['params'][i]['algorithm'], int(grid_model.cv_results_['params'][i]['layers']), int(grid_model.cv_results_['params'][i]['neurons_per_layer']), 
                              grid_model.cv_results_['params'][i]['activation'], grid_model.cv_results_['params'][i]['optimizer'], grid_model.cv_results_['params'][i]['learning_rate'], 
                              horizon, back_steps, grid_model.cv_results_['params'][i]['batch_size'],grid_model.cv_results_['params'][i]['epochs'], dataset, -grid_model.cv_results_['mean_train_score'][i], -grid_model.cv_results_['mean_test_score'][i], grid_model.cv_results_['mean_fit_time'][i]]
results.to_excel('Results1.xlsx', index = False)

print('Best MSE: ', -grid_model.best_score_)
print('Params: ', grid_model.best_params_)

Fitting 1 folds for each of 24 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.3min finished


Best MSE:  235.1434952721103
Params:  {'activation': 'elu', 'algorithm': 'mlp', 'layers': 2, 'learning_rate': 0.005, 'neurons_per_layer': 16, 'optimizer': 'adam'}


#### Random Search

##### GRU, RNN, LSTM, BI LSTM

In [ ]:
iterations = 20

# Hiperparámetros a optimizar. Se pueden seleccionar uno o varios algoritmos!
grid = {'algorithm' : ['gru'], #['gru', 'rnn', 'lstm', 'bilstm'],
            'activation' : ['elu'], #['elu', 'relu', 'selu', ]
            'layers' : list(np.arange(1,6,1)), #min_layers, max_layers (no incluido), step_size
            'neurons_per_layer' : list(np.arange(16, 34, 2)), #min_neurons, max_neurons (no incluido), step_size
            'learning_rate' : list(np.exp(np.linspace(np.log(0.0005),np.log(0.005), 50))), #min_lr, max_lr (incluido), number_of_elements
            'optimizer' : ['adam'], #['adam', 'nadam', 'adadelta', 'adagrad', 'rmsprop']
            'batch_size' : [20],
            'epochs' : [20]}





x_train, y_train, x_valid, y_valid = prepare_data(task = '3d', back_steps = back_steps, horizon = horizon)

# create model
model = KerasRegressor(build_fn = create_model, shape = x_train.shape, verbose = 0)

grid_model = RandomizedSearchCV(estimator = model, param_distributions = grid, n_iter = iterations, verbose = 1, refit = False, n_jobs = -1, pre_dispatch = '2*n_jobs', scoring = 'neg_mean_squared_error',  
                                return_train_score = True, cv = [(list(np.arange(0,x_train.shape[0],1)),list(np.arange(x_train.shape[0], x_train.shape[0]+x_valid.shape[0],1)))])
grid_result = grid_model.fit(np.concatenate((x_train, x_valid)),np.concatenate((y_train, y_valid)))

# save results
results = pd.read_excel('Results1.xlsx')
for i in range(0,len(grid_model.cv_results_['mean_test_score'])):
  results.loc[len(results)] = [grid_model.cv_results_['params'][i]['algorithm'], int(grid_model.cv_results_['params'][i]['layers']), int(grid_model.cv_results_['params'][i]['neurons_per_layer']), 
                              grid_model.cv_results_['params'][i]['activation'], grid_model.cv_results_['params'][i]['optimizer'], grid_model.cv_results_['params'][i]['learning_rate'], 
                              horizon, back_steps, grid_model.cv_results_['params'][i]['batch_size'],grid_model.cv_results_['params'][i]['epochs'], dataset, -grid_model.cv_results_['mean_train_score'][i], -grid_model.cv_results_['mean_test_score'][i], grid_model.cv_results_['mean_fit_time'][i]]
results.to_excel('Results1.xlsx', index = False)

print('Best MSE: ', -grid_model.best_score_)
print('Params: ', grid_model.best_params_)

Fitting 1 folds for each of 4 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  5.4min finished


Best MSE:  186.2228491198188
Params:  {'optimizer': 'adam', 'neurons_per_layer': 22, 'learning_rate': 0.0020474575311902133, 'layers': 1, 'algorithm': 'gru', 'activation': 'elu'}


##### MLP

In [ ]:
iterations = 100

# Hiperparámetros a optimizar. Se pueden seleccionar uno o varios algoritmos!
grid = {'algorithm' : ['mlp'],
            'activation' : ['elu'], #['elu', 'relu', 'selu', ]
            'layers' : list(np.arange(1,6,1)), #min_layers, max_layers (no incluido), step_size
            'neurons_per_layer' : list(np.arange(16, 34, 2)), #min_neurons, max_neurons (no incluido), step_size
            'learning_rate' : list(np.exp(np.linspace(np.log(0.0005),np.log(0.005), 50))), #min_lr, max_lr (incluido), number_of_elements
            'optimizer' : ['adam'], #['adam', 'nadam', 'adadelta', 'adagrad', 'rmsprop']
            'batch_size' : [20],
            'epochs' : [20]}




x_train, y_train, x_valid, y_valid = prepare_data(task = '2d', back_steps = back_steps, horizon = horizon)

# create model
model = KerasRegressor(build_fn = create_model, shape = x_train.shape, verbose = 0)

grid_model = RandomizedSearchCV(estimator = model, param_distributions = grid, n_iter = iterations, verbose = 1, refit = False, n_jobs = -1, pre_dispatch = '2*n_jobs', scoring = 'neg_mean_squared_error', 
                                return_train_score = True, cv = [(list(np.arange(0,x_train.shape[0],1)),list(np.arange(x_train.shape[0], x_train.shape[0]+x_valid.shape[0],1)))])
grid_result = grid_model.fit(np.concatenate((x_train, x_valid)),np.concatenate((y_train, y_valid)))

# save results
results = pd.read_excel('Results1.xlsx')
for i in range(0,len(grid_model.cv_results_['mean_test_score'])):
  results.loc[len(results)] = [grid_model.cv_results_['params'][i]['algorithm'], int(grid_model.cv_results_['params'][i]['layers']), int(grid_model.cv_results_['params'][i]['neurons_per_layer']), 
                              grid_model.cv_results_['params'][i]['activation'], grid_model.cv_results_['params'][i]['optimizer'], grid_model.cv_results_['params'][i]['learning_rate'], 
                              horizon, back_steps, grid_model.cv_results_['params'][i]['batch_size'],grid_model.cv_results_['params'][i]['epochs'], dataset, -grid_model.cv_results_['mean_train_score'][i], -grid_model.cv_results_['mean_test_score'][i], grid_model.cv_results_['mean_fit_time'][i]]
results.to_excel('Results1.xlsx', index = False)

print('Best MSE: ', -grid_model.best_score_)
print('Params: ', grid_model.best_params_)

Fitting 1 folds for each of 10 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   40.1s finished


Best MSE:  233.19771857050605
Params:  {'optimizer': 'adam', 'neurons_per_layer': 32, 'learning_rate': 0.0021459671300643893, 'layers': 2, 'epochs': 20, 'batch_size': 20, 'algorithm': 'mlp', 'activation': 'elu'}


#### Bayesian Optimization

##### MLP, GRU, RNN, LSTM, BI LSTM

In [ ]:
from hyperopt import hp, fmin, tpe, space_eval, Trials

max_evals = 100

# Hiperparámetros a optimizar. Se pueden seleccionar uno o varios algoritmos!
space = {'algorithm' : hp.choice('algorithm', ['mlp', 'gru', 'rnn', 'lstm', 'bilstm']), #['mlp', 'gru', 'rnn', 'lstm', 'bilstm'],
            'activation' : hp.choice('activation',['elu', 'selu']), #['elu', 'relu', 'selu', ]
            'layers' : hp.quniform('layers', 1, 5, 1), #min_layers, max_layers, step_size
            'neurons_per_layer' : hp.quniform('neurons_per_layer', 16, 64, 2), #min_neurons, max_neurons, step_size
            'learning_rate' : hp.loguniform('learning_rate', np.log(0.00005), np.log(0.005)), #min_lr, max_lr
            'optimizer' : hp.choice('optimizer',['adam']), #['adam', 'nadam', 'adadelta', 'adagrad', 'rmsprop']
            'batch_size' : hp.choice('batch_size',[20]),
            'epochs' : hp.choice('epochs',[20])}
         
# Se puede incluir en las iteraciones el dropout

def objective(params):
    try:
      set_seed()

      # Prepare data
      if params['algorithm'] == 'mlp':
        x_train, y_train, x_valid, y_valid = prepare_data(task = '2d', back_steps = back_steps, horizon = horizon)
      else:
        x_train, y_train, x_valid, y_valid = prepare_data(task = '3d', back_steps = back_steps, horizon = horizon)

      model = create_model(algorithm = params['algorithm'], layers = int(params['layers']), neurons_per_layer = int(params['neurons_per_layer']),
                           activation = params['activation'], optimizer = params['optimizer'], learning_rate = params['learning_rate'], shape = x_train.shape)

      # Fit model on the dataset
      start_time = time.time()
      history = model.fit(x_train, y_train, epochs = int(params['epochs']), batch_size = int(params['batch_size']), validation_data = None, verbose = 0, shuffle = False)
      training_time = time.time() - start_time

      # Evaluate model
      y_pred = model.predict(x_valid)
      mse = sklearn.metrics.mean_squared_error(y_valid, y_pred)

      results = pd.read_excel('Results.xlsx')
      results.loc[len(results)] = [params['algorithm'], int(params['layers']), int(params['neurons_per_layer']), params['activation'], 
                                   params['optimizer'], params['learning_rate'], horizon, back_steps, int(params['batch_size']), int(params['epochs']), dataset, history.history['loss'][-1], mse, training_time]
      results.to_excel('Results.xlsx', index = False)

    except:
      mse = 1000000.0     
      results = pd.read_excel('Results.xlsx')
      results.loc[len(results)] = [params['algorithm'], int(params['layers']), int(params['neurons_per_layer']), params['activation'], 
                                   params['optimizer'], params['learning_rate'], horizon, back_steps, int(params['batch_size']), int(params['epochs']), dataset, 'error', 'error', 'error']
      results.to_excel('Results.xlsx', index = False)

    return mse     #Minimize

trials = Trials()
best = fmin(fn = objective, space = space, max_evals = max_evals, algo = tpe.suggest, trials = trials)
print('\n', best)

100%|██████████| 5/5 [01:07<00:00, 13.48s/it, best loss: 210.89283989600403]

 {'activation': 1, 'algorithm': 2, 'batch_size': 0, 'epochs': 0, 'layers': 2.0, 'learning_rate': 0.0028969581951935625, 'neurons_per_layer': 16.0, 'optimizer': 0}


#### Genetic Algorithms

##### MLP, GRU, RNN, LSTM, BI LSTM

In [ ]:
def fit_model(algorithm, activation, optimizer, learning_rate, layers, neurons_per_layer, batch_size, epochs):

    set_seed()

    if algorithm == 'mlp':
      x_train, y_train, x_valid, y_valid = prepare_data(task = '2d', back_steps = back_steps, horizon = horizon)
    else:
      x_train, y_train, x_valid, y_valid = prepare_data(task = '3d', back_steps = back_steps, horizon = horizon)
    
    model = create_model(algorithm = algorithm, activation = activation, optimizer = optimizer, learning_rate = learning_rate, 
                         layers = layers, neurons_per_layer = neurons_per_layer, shape = x_train.shape)
    
    start_time = time.time()
    history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = None, verbose = 0, shuffle = False)
    training_time = time.time() - start_time
    
    return model, history.history['loss'][-1], training_time


class Network():
    def __init__(self):
        #self._epochs = np.random.randint(10, 20)
        #self._batch_size = np.random.randint(10, 25)
        self._epochs = epochs
        self._batch_size = batch_size

        self._algorithm = rn.choice(['mlp']) #['mlp', 'gru', 'rnn', 'lstm', 'bilstm']

        self._layers = np.random.randint(1, 6)
        self._neurons_per_layer = 10 + 2 * np.random.randint(1, 28)

        self._activation = rn.choice(['elu']) #['relu', 'elu', 'selu']
        self._optimizer = rn.choice(['adam']) #['adam', 'nadam', 'adadelta', 'adagrad', 'rmsprop']
        self._learning_rate = np.exp(np.log(0.0005) + np.random.rand() * (np.log(0.005) - np.log(0.0005)))

        self._mse = 1000000

    def init_hyperparams(self):
        hyperparams = {
            'epochs': self._epochs,
            'batch_size': self._batch_size,
            'algorithm': self._algorithm,
            'layers': self._layers,
            'neurons_per_layer': self._neurons_per_layer,
            'activation': self._activation,
            'optimizer': self._optimizer,
            'learning_rate': self._learning_rate
        }
        return hyperparams

def init_networks(population):
    return [Network() for _ in range(population)]

def fitness_network(network):

    hyperparams = network.init_hyperparams()
    epochs = hyperparams['epochs']
    batch_size = hyperparams['batch_size']
    algorithm = hyperparams['algorithm']
    layers = hyperparams['layers']
    neurons_per_layer = hyperparams['neurons_per_layer']
    activation = hyperparams['activation']
    optimizer = hyperparams['optimizer']
    learning_rate = hyperparams['learning_rate']
    
    try:
        model, training_error, training_time = fit_model(algorithm = algorithm, activation = activation, optimizer = optimizer, learning_rate = learning_rate,
                                                          layers = layers, neurons_per_layer = neurons_per_layer, batch_size = batch_size, epochs = epochs)
        
        if algorithm == 'mlp':
          x_train, y_train, x_valid, y_valid = prepare_data(task = '2d', back_steps = back_steps, horizon = horizon)
        else:
          x_train, y_train, x_valid, y_valid = prepare_data(task = '3d', back_steps = back_steps, horizon = horizon)

        y_pred = model.predict(x_valid)
        mse = sklearn.metrics.mean_squared_error(y_valid, y_pred)

        results = pd.read_excel('Results1.xlsx')
        results.loc[len(results)] = [algorithm, int(layers), int(neurons_per_layer), activation, optimizer, learning_rate, horizon, back_steps, batch_size, epochs, dataset,
                                      training_error, mse, training_time]
        results.to_excel('Results1.xlsx', index = False)

        network._mse = mse
        print ('MSE: {}'.format(network._mse))

    except:
        network._mse = 1000000
        results = pd.read_excel('Results1.xlsx')
        results.loc[len(results)] = [algorithm, int(layers), int(neurons_per_layer), activation, optimizer, learning_rate, horizon, back_steps, batch_size, epochs, dataset,
                                      'error', 'error', 'error']
        results.to_excel('Results1.xlsx', index = False)
        print ('Error')

    return network

def fitness(networks, parallelization = False):
    if parallelization:
      n_networks = Parallel(n_jobs=-1)(delayed(fitness_network)(network) for network in networks)
      for network in n_networks:
        print ('MSE: {}'.format(network._mse))
    else:
      n_networks = []
      for network in networks:
        n_networks.append(fitness_network(network))
    return n_networks

def selection(networks):
    networks = sorted(networks, key=lambda network: network._mse, reverse=False)
    networks = networks[:max(int(0.2 * len(networks)), 2)]

    return networks

def crossover(networks):
    offspring = []
    for _ in range(int((population - len(networks)) / 2)):
        parent1 = rn.choice(networks)
        parent2 = rn.choice(networks)
        child1 = Network()
        child2 = Network()

        # Crossing over parent hyper-params
        child1._epochs = parent2._epochs #max(int(3*parent1._epochs/4 + parent2._epochs/4),10)
        child2._epochs = parent1._epochs #max(int(parent1._epochs/4 + 3*parent2._epochs/4),10)

        child1._batch_size = parent2._batch_size #max(int(parent1._batch_size/4) + 3*parent2._batch_size/4),10)
        child2._batch_size = parent1._batch_size #max(int(3*parent1._batch_size/4) + parent2._batch_size/4),10)

        child1._algorithm = parent2._algorithm
        child2._algorithm = parent1._algorithm        
        
        child1._layers = rn.choice([parent1._layers, parent2._layers]) #max(int(3*parent1._layers/4 + parent2._layers/4),1)
        child2._layers = rn.choice([parent1._layers, parent2._layers]) #max(int(parent1._layers/4 + 3*parent2._layers/4),1)

        child1._neurons_per_layer = max(int(parent1._neurons_per_layer/4 + 3*parent2._neurons_per_layer/4), 8)
        child2._neurons_per_layer = max(int(3*parent1._neurons_per_layer/4 + parent2._neurons_per_layer/4), 8)

        child1._activation = parent2._activation
        child2._activation = parent1._activation

        child1._optimizer = parent1._optimizer
        child2._optimizer = parent2._optimizer

        child1._learning_rate = parent1._learning_rate
        child2._learning_rate = parent2._learning_rate

        offspring.append(child1)
        offspring.append(child2)

    networks.extend(offspring)

    return networks

def mutate(networks):
    for network in networks:
        if np.random.uniform(0, 1) <= 0.1:
            #network._epochs += 5 * np.random.randint(0,3)
            #network._batch_size += 5 * np.random.randint(0,3)
            network._layers += rn.choice([0,1,1,2])
            network._neurons_per_layer += 2 * np.random.randint(0,9)
            network._learning_rate *= (0.8 + 0.4 * np.random.rand())

    return networks

In [ ]:
batch_size = 20
epochs = 20
threshold = 170
generations = 4
population = 20
parallelization = True #Colab solo tiene 2 cores

networks = init_networks(population)
best_mse = np.inf
best_network = None

for gen in range(generations):
    print ('\nGeneration {}'.format(gen+1))
    networks = fitness(networks, parallelization)
    networks = selection(networks)
    networks = crossover(networks)
    networks = mutate(networks)

    for network in networks:
        if network._mse < best_mse:
          best_network = network
          best_mse = network._mse 

    if best_mse < threshold:
        print ('\nThreshold met')
        break

   
print ('\n' + str(best_network.init_hyperparams()))
print ('Best MSE: {}'.format(best_network._mse))


Generation 1
MSE: 332.81160578595086
MSE: 270.8185266855574
MSE: 293.27991452925016
MSE: 322.0624300076036

Generation 2
MSE: 255.86785873472246
MSE: 262.83359241866077
MSE: 248.50972829628066
MSE: 294.54793611585416

{'epochs': 20, 'batch_size': 20, 'algorithm': 'mlp', 'layers': 6, 'neurons_per_layer': 34, 'activation': 'elu', 'optimizer': 'adam', 'learning_rate': 0.0006402269437271258}
Best MSE: 248.50972829628066


In [ ]:
networks[3]._mse

1000000